# Joint conf-call report #2

In [1]:
import datetime
print(str(datetime.datetime.today()))

2018-05-04 11:49:43.104378


_For this second report I am experimenting a new way to fastly deploy updates about the project: **Jupyter notebook**. In this stage of the project, I'd like to share more code snippets than I would in the final thesis, hence markdown support and live execution of code snippets turned out to be very useful for such purpose. Actually the two main reason why I decided to move from raw LaTeX to here are: (1) Jupyter does it faster, (2) it converts everything to LaTeX (I won't be required to make any effort to include everything from here to my LaTeX thesis document). Whether, for any reason, this method will turn out to be ineffective or time-expensive, then I will drop it out._

_You will notice changes in style, format and even contents (like change logs and other stuffs that may appear useless or too much detailed), this necessity arises since I need to keep track of the internship's history and I foresee that within few months the commit log on Github will get a bit messy and too much wide to be relied on, instead these reports are the easiest way to fulfill such task._

# Change log summary

## Training set generator function
Now domains of all samples' components are centered at the same value $c$. Such domains are determined as follows.

Is defined $K = \{k_1,...k_m\}$ where $k_i$ is the domain radius of variable $x_i$.
Let $x=(x_1,...,x_m) \in X$ be a sample of the training set, then the domain of $x_i$ is $D(x_i) = [c-k_i, c+k_i]$.


```python
def sample_from_function(n_samples, n_features, func, domain_radius=0.5, domain_center=0.5, error_mean=0, error_std_dev=1):
    X = []
    y = np.zeros(n_samples)
    w = np.ones(n_features)
    K = np.random.uniform(domain_center - domain_radius, domain_center + domain_radius, n_features)

    for i in range(n_samples):
        x = np.zeros(n_features)
        for j in range(n_features):
            x[j] = np.random.uniform(-K[j] + domain_radius, K[j] + domain_radius)
        X.append(x)
        y[i] = func(x, w) + np.random.normal(error_mean, error_std_dev)

return np.array(X), y
```

**Example**. Detailed execution of `sample_from_function` with the following parameters:
- `n_samples = 8` : amount of samples in the training set;
- `n_features = 4` : number of features, e.g. size of each sample's array;
- `func` : classic linear function $f(\vec{x},\vec{w})=\sum_{j=1}^{m} x_j w_j$ where $m$ is the size of $x$ (and $w$), so, for instance, $f((2,3),(4,1))=2*4+3*1=11$;
- `domain_radius = 5`;
- `domain_center = 0`;
- `error_mean = 0` and `error_std_dev = 1`: the error (noise) is normally distributed with $\mu = 0$ and $\sigma = 1$.

In [2]:
import numpy as np
import pprint
n_samples = 8
n_features = 4
func = lambda _X,_w : _X.dot(_w)
domain_center = 0
domain_radius = 5
error_mean = 0
error_std_dev = 1
X = []
y = np.zeros(n_samples)
w = np.ones(n_features)
K = np.random.uniform(domain_center - domain_radius, domain_center + domain_radius, n_features)

print(K)

[ 4.00713793 -0.33902602  0.13068124  1.6246818 ]


These are the centers of variables $x_1, x_2, x_3$ and $x_4$ respectively.

In [3]:
for i in range(n_samples):
    x = np.zeros(n_features)
    for j in range(n_features):
        x[j] = np.random.uniform(-K[j] + domain_radius, K[j] + domain_radius)
    X.append(x)
    y[i] = func(x, w) + np.random.normal(error_mean, error_std_dev)
print("X = \n" + pprint.PrettyPrinter(indent=4).pformat(X))
print("y = " + pprint.PrettyPrinter(indent=4).pformat(y))

X = 
[   array([1.14678659, 4.72191439, 4.88513418, 4.385389  ]),
    array([6.3692572 , 5.30138619, 5.01406369, 5.63875345]),
    array([4.67455441, 5.21725698, 5.02592469, 5.45187453]),
    array([8.22688716, 5.23973922, 5.01244772, 5.39299295]),
    array([6.31345814, 4.74067851, 5.02559019, 5.67637608]),
    array([1.11971923, 4.88378811, 4.88175249, 6.00179764]),
    array([2.11584329, 4.96572785, 5.04728619, 5.44922452]),
    array([7.86286299, 4.94175031, 5.12274525, 4.64251712])]
y = array([15.78801252, 22.09363563, 19.78453008, 24.20968846, 22.6499015 ,
       15.91160444, 17.96298163, 24.32308662])


Above is printed the training set $\mathcal{X} = (X,y)$.

## New metric Real Mean Squared Error (RMSE)
The training set is in the form of a pair $(X,y)$ where $y_i$ is the value the target function is supposed to yield for the input $x_i$, actually, whether a noise exists in the training set, then $y_i$ differs from the real value $\tilde{y}_i$ by a gap that, in our case, is normally distributed (with mean = `error_mean` and standard deviation = `error_std_dev`). Whereas the training set is generated with fully control over all parameters, we know either the perturbed value $y_i$ either the real value $\tilde{y}_i=\mathbb{1} x_i=\sum_{j=1}^{m}x_{ij}$, so we have an additional information to take into account in order to study behaviour of different models.

While the mean squared error
$$MSE = \frac{1}{N} \sum_{i=1}^{n} (y_i - \hat{y}_i) (\hat{y}_i)'$$
regards $y_i$, the _real mean squared error_ (RMSE) concerns $\tilde{y}_i$, hence it's defined as
$$RMSE = \frac{1}{N} \sum_{i=1}^{n} (\tilde{y}_i - \hat{y}_i) (\hat{y}_i)'.$$

By comparing these two metrics one can understand whether and how much the prediction model suffers from noise fitting, e.g. when the model adapts itself much more on the noise rather than on the provided target function value.

Obviously, noiseless training sets lead the MSE to be equal to the RMSE.

## Computing the error over the whole training set
After each iteration each node computes a set of metrics taking into account its local model and knowledge, so each node keeps track of the history of its weight vector, MAE (mean absolute error), RMAE (to be implemented yet), MSE, RMSE.

Previously I computed the error as the mean of the MSE of each node. A way that, at first glance, could seem reasonable, but, actually, it is not: such MSE is not computed with one weight vector over the entire training set, indeed if someone asked for the weight vector which had produced such result, then we would not have a value to provide they with.
That's why the correct way to compute the global metrics is to retrieve from each node $k$ its local model $\vec{w}_{k}$, compute the global model $\vec{w} = \frac{1}{K}\sum_{k=1}^{K}\vec{w}_k$ and then compute metrics taking into account $\vec{w}$ along with the whole training set.


## Test description
Since there are several parameters to set up in order to run a test, I have excluded to pass them from the command line (by running the program in a way like `$ python main.py [parameters]`), instead they're are set directly inside the script `main.py`. Leave aside for a while the system and training task setup, there are many other settings which ensure control over the test's execution and outputs. Without deepening their implementation, when a new test is run, the simulator creates a new folder `/$TEST_NAME` in `/test_log` that contains:
- a `/plot` folder with all plots images;
- all global logs of the simulation for each topology;
- a descriptor file that reports the detailed parameters' values used for the test;
- a serialization of the setup that can be used to run again the same simulation.

**Example of a test folder tree**.
```
./test_log
└── /$TEST_NAME
    ├── /plot
    │   ├── iter_time.png
    │   ├── mse_iter.png
    │   ├── mse_time.png
    │   ├── real-mse_iter.png
    │   └── real-mse_time.png
    ├── clique_global_mean_squared_error_log
    ├── clique_global_real_mean_squared_error_log
    ├── clique_iterations_time_log
    ├── cycle_global_mean_squared_error_log
    ├── cycle_global_real_mean_squared_error_log
    ├── cycle_iterations_time_log
    ├── diagonal_global_mean_squared_error_log
    ├── diagonal_global_real_mean_squared_error_log
    ├── diagonal_iterations_time_log
    ├── diam-expander_global_mean_squared_error_log
    ├── diam-expander_global_real_mean_squared_error_log
    ├── diam-expander_iterations_time_log
    ├── root-expander_global_mean_squared_error_log
    ├── root-expander_global_real_mean_squared_error_log
    ├── root-expander_iterations_time_log
    ├── .descriptor.txt
    └── .setup.pkl
```

### Test descriptor file `.descriptor.txt`
Below how the descriptor appears. Although each parameter is provided with a short description embedded within a comment, it doesn't matter if some or most of them won't be immediately understandable, however simply take a fast look at this code snippet to realize how many parameters the system let us customize. Remind that such **descriptor file is auto-generated** right after a test has started.

```python
# >>> Test Descriptor File
Title: 'test'
Date: '2018-04-26 11:28:39.116108'
Summary: '-'

### BEGIN SETUP ###
n = 10  # amount of computational nodes
seed = 1524734919  # random initial seed

# adjacency matrices for each graph
graphs = {
    'clique': np.array([
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]),
    'cycle': np.array([
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
    'diagonal': np.array([
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
    'diam-expander': np.array([
        [1., 1., 0., 0., 0., 1., 0., 0., 0., 1.],
        [1., 1., 1., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 1., 1., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 1., 1., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0., 0., 1.],
        [1., 0., 0., 0., 1., 1., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 1., 1., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 1., 1., 1., 0.],
        [0., 0., 0., 1., 0., 0., 0., 1., 1., 1.],
        [1., 0., 0., 0., 1., 0., 0., 0., 1., 1.]]),
    'root-expander': np.array([
        [1., 1., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 1., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 1., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1., 0., 1.],
        [1., 0., 0., 0., 0., 0., 0., 1., 1., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 1., 1.],
        [1., 0., 1., 0., 0., 0., 0., 0., 0., 1.]]),
    'star': np.array([
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]
    )}

# TRAINING SET SETUP
n_samples = 100000
n_features = 100

# function used to generate the training set
sample_function = <function LinearYHatFunction.f at 0x7f13fa4d00d0>
domain_radius = 5
domain_center = 0
error_mean = 0
error_std_dev = 1

# CLUSTER SETUP
max_iter = None  # upon reaching max_iter iterations the cluster will stop
max_time = 100000  # the same for the time 
yhat = <class 'src.mltoolbox.LinearYHatFunction'>  # function to learn the parameters of
method = 'stochastic'
batch_size = 20  # matters only for the batch gradient method
activation_func = None  # to specify for classification training task for instance
loss = <class 'src.mltoolbox.SquaredLossFunction'>  # loss function
penalty = 'l2'  # not used yet
epsilon = 0.01  # error goodness threshold
alpha = 1e-06  # alpha constant
learning_rate = 'constant'  # type of learning rate, not used yet
metrics = 'all'  # specify which metrics the system may take into account
alt_metrics = False  # if True then the MSE will be computed as it happened earlier
shuffle = True  # whether the cluster should shuffle the training set
verbose = False  # not used yet
```

# Simulation useful informations
In this section are listed some useful details in order to doubtless understand further simulation outputs.

## Meaning of _iteration_
Each node owns a local field `iteration`. The starting value is 0. Actually "iteration equal to $t$" means that the weight vector has been updated $t$ times. A single update of the weight $w_{t+i} = w_t - \alpha \nabla Err$ implies the iteration to increase by one from $t$ to $t+1$.
- "Node $i$ is at iteration $t$" means it has already computed the $t$-th step of the gradient descent method, thus $w_t$ is its the most recent weight vector owned by $i$.
- "Node $i$ computes/performs iteration $t$" means that $i$ is performing the update $w_{t} = w_{t-1} - \alpha \nabla Err$.

The _cluster_ model (the object that supervises the distributed simulation) has an `iteration` field too, it is equal to the fewest iteration among all nodes. `cluster.iteration` being equal to $t$ means that each node has already performed at least $t$ updates of its local $w_t$ but at least one node has not computed $w_{t+1}$ yet, thus a global $w_t$ can be retrieved (and used to compute metrics for instance).

## Single iteration time
> **`time.perf_counter()`**

> Return the value (in fractional seconds) of a performance counter, i.e. a clock with the highest available resolution to measure a short duration. It does include time elapsed during sleep and is **system-wide**. The reference point of the returned value is undefined, so that only the difference between the results of consecutive calls is valid.

> *(from the [Python Standard Library Documentation](https://docs.python.org/3/library/time.html#time.perf_counter))*

`time.perf_counter()` is the value Python developers suggest to use in order to make performance evaluation of python algorithms, but, obviously, since it is taken directly from the processor, it is *system-wide*, thus depends on too many unknown and unmanageable variable conditions (i.e. other system processes, OS processes' managing system, etc.).

It's clear why the time taken by a node to perform a single iteration computation (e.g. gradient descent weights update) cannot be retrieved considering such clock, so the function below isn't reliable at all:

```python
def unreliable_gradient_step(self):
    """
    Perform a single step of the gradient descent method.
    :return: a list containing [clock_before, clock_after] w.r.t. the computation
    """
    # useful vars for estimate the time taken by the computation
    t0 = self.local_clock
    # get the processor time counter before the computation starts
    c0 = time.perf_counter()

    ## STEP COMPUTATION BEGIN
    # avg internal self.W vector with W incoming from dependencies
    if self.iteration > 0:
        self.avg_weight_with_dependencies()
    # compute the gradient descent step
    self.training_task.step()
    # broadcast the obtained value to all node's recipients
    self.broadcast_weight_to_recipients()
    ## STEP COMPUTATION END
        
    # get the processor counter after the computation has ended
    cf = time.perf_counter()
    # computes the clock when the computation has finished
    tf = t0 + cf - c0
        
    # update the local_clock
    self.local_clock = tf
    self.iteration += 1

    return [t0, tf]
```

Instead it is more convenient and reasonable to consider the following function:

```python
def reliable_gradient_step(self):
    """
    Perform a single step of the gradient descent method.
    :return: a list containing [clock_before, clock_after] w.r.t. the computation
    """
    # useful vars for estimate the time taken by the computation
    t0 = self.local_clock

    # avg internal self.W vector with W incoming from dependencies
    if self.iteration > 0:
        self.avg_weight_with_dependencies()

    # compute the gradient descent step
    self.training_task.step()

    # broadcast the obtained value to all node's recipients
    self.broadcast_weight_to_recipients()

    # get the counter after the computation has ended
    # get the time from an exponential with lambda parameter equal to 1
    # whereas lambda = 1 / mean_time, so mean_time = 1
    dt = random.expovariate(1)

    # computes the clock when the computation has finished
    tf = t0 + dt
    # update the local_clock
    self.local_clock = tf

    self.iteration += 1
    self.log.append(self.local_clock)

    return [t0, tf]
```

The time taken by a node to perform a single gradient step is handled by a random variable $X \sim Exp(\lambda)$ (e.g. exponentially distributed with parameter $\lambda$) where $\mathbb{E}[X] = \frac{1}{\lambda}$. At the moment $\lambda$ is fixed to $1$, so, in this case, the mean time is equal to one too.

_It is not mandatory for you to know more details about how the system actually works._

# Previous tests - 002 (those made until 25/04/18)
Outdated, not reported here.

# New tests - 003 (since 26/04/18)
## Main changes from earlier tests
1. The number of computational units has been fixed to $n=10$;
2. `diam-expander` has lost one edge per node. It is now defined as
    $$G=(V=\{0,...n-1\},E=\{(i, i+1\text{ mod }n), (i, i+\frac{n}{2}\text{ mod }n) : i \in [n]\});$$
    
    **Example of a diameter expander with 6 nodes.**
    
    ![Diam-expander graph](media/img/graph-samples/diam-expander.png)
3. a new dependency graph `root-expander` has been considered
    $$G=(V=\{0,...n-1\},E=\{(i, i+1\text{ mod }n), (i, i+\sqrt{n}\text{ mod }n) : i \in [n]\});$$
    
    **Example of a root expander with 6 nodes.**
    
    ![Root-expander graph](media/img/graph-samples/root-expander.png)
4. at the moment we will focus on testing the classic GD method. Right after obtaining some results for GD, we will test BGD (batch gradient descent) and SGD (stochastic gradient descent) as well.

## Relevant tests fixed parameters
```python
n = 10
seed = str(time.time()) # timestamp
graphs = {
    # cycle, clique, root-expander, diam-expander, diagonal adjacency matrices
}

# TRAINING SET SETUP
n_features = 100
yhat = <class 'src.mltoolbox.LinearYHatFunction'>
domain_radius = 5
domain_center = 0
error_mean = 0
error_std_dev = 1

# CLUSTER SETUP
sample_function = <function LinearYHatFunction.f at 0x7f13fa4d00d0>
max_iter = None
max_time = 10000
method = 'classic'
loss = <class 'src.mltoolbox.SquaredLossFunction'>
penalty = l2
epsilon = 0.01
alpha = 1e-06
learning_rate = constant
metrics = all
alt_metrics = False
shuffle = True
verbose = False
```

The parameter we will focus particularly on and the only which will vary is `n_samples`.

## 100 samples GD test
`n_samples = 100`.

### 100 sample GD test - Iterations over time
![Iterations over time](media/img/tests/test_003_100samples_classic/1_iter_time.png)
This trend will be a constant through all future tests: the less the nodes' connections the fastest the system can advance in iterations.

As expected, in diagonal topology, since nodes never wait still and since the mean of the time taken by each iteration is equal to 1, then such topology always achieves an amount of iterations approximately equal to the time of the simulation. This plot makes us notice that the amount of iteration achieved by each topology depends, in some way, on the graph's degree (e.g. number of dependency established among nodes). Further, we would like to study in depth this intuition.

### 100 sample GD test - MSE over iterations
![MSE over iter](media/img/tests/test_003_100samples_classic/2_mse_iter.png)
Due to restricted training set size, a single node doesn't own enough samples to achieve a good model without communicating with other nodes, therefore diagonal topology performs far worse than the others, which in turn behave almost equally.

### 100 samples GD test - RMSE over iterations
![RMSE over iter](media/img/tests/test_003_100samples_classic/2_real-mse_iter.png)
RMSE/iter plot doesn't suggest anything more than MSE/iter already does.

### 100 samples GD test - MSE over time
![MSE over time](media/img/tests/test_003_100samples_classic/3_mse_time.png)

### 100 samples GD test - RMSE over time
![RMSE over time](media/img/tests/test_003_100samples_classic/3_real-mse_time.png)
![RMSE over time_zoom](media/img/tests/test_003_100samples_classic/3_real-mse_time_zoom.png)

As already seen in previous plots for this test, diagonal topology, unlike all the others, never achieves a good prediction model. 

The cycle turns out to be the fastest.

2-degree graphs plots (root expander and diameter expander) are very close.


_The same test with different seeds has led to almost identical results._

## 1k samples GD test
`n_samples = 1000`.

### 1k samples GD test - iterations over time
![Iterations over time](media/img/tests/test_003_1ksamples_classic/1_iter_time.png)
Almost the same result as in 100 samples test.

### 1k samples GD test - MSE over iterations
![MSE over iterations](media/img/tests/test_003_1ksamples_classic/2_mse_iter.png)
![MSE over iterations zoom](media/img/tests/test_003_1ksamples_classic/2_mse_iter_zoom.png)

Now, since the amount of samples in the training set is equal to 1000, each node $i$ owns a subsample $\mathcal{X}_i \subset \mathcal{X}$ such that $|\mathcal{X}_i| = 100$ (when for $|\mathcal{X}|=100$, $|\mathcal{X}_i|=10$). Such condition seems enough for the diagonal topology to perform similarly as those with communications among nodes.

### 1k samples GD test - RMSE over iterations
![RMSE over iterations](media/img/tests/test_003_1ksamples_classic/2_real-mse_iter.png)
![RMSE over iterations zoom](media/img/tests/test_003_1ksamples_classic/2_real-mse_iter_zoom.png)
By comparing MSE/iter and RMSE/iter, we notice that diagonal topology suffers noise more than other topologies do.

### 1k samples GD test - MSE over time
![MSE over time](media/img/tests/test_003_1ksamples_classic/3_mse_time.png)

### 1k samples GD test - RMSE over time
![RMSE over time](media/img/tests/test_003_1ksamples_classic/3_real-mse_time.png)
![RMSE over time_zoom](media/img/tests/test_003_1ksamples_classic/3_real-mse_time_zoom.png)

Despite the behaviour of topologies, with respect to **convergence speed**, is almost the same as the one observed in the _100 samples test MSE/time plot_, now diagonal topology doesn't behave so bad anymore (its error differs only by 1 from other topologies). 

_The same test with different seeds has led to almost identical results._

## 10k samples test
`n_samples = 10000`.

### 10k samples GD test - iterations over time
![Iterations over time](media/img/tests/test_003_10ksamples_classic/1_iter_time.png)

### 10k samples GD test - MSE over iterations
![MSE over iterations](media/img/tests/test_003_10ksamples_classic/2_mse_iter.png)

### 10k samples GD test - RMSE over iterations
![RMSE over iterations](media/img/tests/test_003_10ksamples_classic/2_real-mse_iter.png)
![RMSE over iterations zoom](media/img/tests/test_003_10ksamples_classic/2_real-mse_iter_zoom.png)

### 10k samples GD test - MSE over time
![MSE over time](media/img/tests/test_003_10ksamples_classic/3_mse_time.png)
![MSE over time_zoom](media/img/tests/test_003_10ksamples_classic/3_mse_time_zoom.png)

### 10k samples GD test - RMSE over time
![RMSE over time](media/img/tests/test_003_10ksamples_classic/3_real-mse_time.png)
![RMSE over time_zoom](media/img/tests/test_003_10ksamples_classic/3_real-mse_time_zoom.png)

With $10000$ samples diagonal outperforms other topologies with regard to either MSE/time and RMSE/time plots.

_The same test with different seeds has led to almost identical results._

## 100k samples GD test
`n_samples = 100000`.

### 100k samples GD test - iterations over time
![Iterations over time](media/img/tests/test_003_100ksamples_classic/1_iter_time.png)

### 100k samples GD test - MSE over iterations
![MSE over iterations](media/img/tests/test_003_100ksamples_classic/2_mse_iter.png)

### 100k samples GD test - RMSE over iterations
![RMSE over iterations](media/img/tests/test_003_100ksamples_classic/2_real-mse_iter.png)
![RMSE over iterations zoom](media/img/tests/test_003_100ksamples_classic/2_real-mse_iter_zoom.png)

### 100k samples GD test - MSE over time
![MSE over time](media/img/tests/test_003_100ksamples_classic/3_mse_time.png)
![MSE over time_zoom](media/img/tests/test_003_100ksamples_classic/3_mse_time_zoom.png)

### 100k samples GD test - RMSE over time
![RMSE over time](media/img/tests/test_003_100ksamples_classic/3_real-mse_time.png)
![RMSE over time_zoom](media/img/tests/test_003_100ksamples_classic/3_real-mse_time_zoom.png)

_The same test with different seeds has led to almost identical results._

While the training set gets bigger, diagonal topology turns out to be by far the fastest. I would like to make some tests with different kinds of training sets to assess whether this results are strongly related to this training set morphology or can be extended to more general cases. 

---
Below tests with 100 and 1000 samples repeated with `method='stochastic'` instead of `classic` for the gradient descent.

## 100 samples SGD test

### 100 samples SGD test - iterations over time
![Iterations over time](media/img/tests/test_003_100samples_stochastic/1_iter_time.png)

### 100 samples SGD test - MSE over iterations
![MSE over iterations](media/img/tests/test_003_100samples_stochastic/2_mse_iter.png)
![MSE over iterations](media/img/tests/test_003_100samples_stochastic/2_mse_iter_zoom.png)

### 100 samples SGD test - RMSE over iterations
![RMSE over iterations](media/img/tests/test_003_100samples_stochastic/2_real-mse_iter.png)
![RMSE over iterations zoom](media/img/tests/test_003_100samples_stochastic/2_real-mse_iter_zoom.png)

### 100 samples SGD test - MSE over time
![MSE over time](media/img/tests/test_003_100samples_stochastic/3_mse_time.png)

### 100 samples SGD test - RMSE over time
![RMSE over time](media/img/tests/test_003_100samples_stochastic/3_real-mse_time.png)

## 1k samples SGD test

### 1k samples SGD test - iterations over time
![Iterations over time](media/img/tests/test_003_1ksamples_stochastic/1_iter_time.png)

### 1k samples SGD test - MSE over iterations
![MSE over iterations](media/img/tests/test_003_1ksamples_stochastic/2_mse_iter.png)
![MSE over iterations](media/img/tests/test_003_1ksamples_stochastic/2_mse_iter_zoom.png)

### 1k samples SGD test - RMSE over iterations
![RMSE over iterations](media/img/tests/test_003_1ksamples_stochastic/2_real-mse_iter.png)
![RMSE over iterations zoom](media/img/tests/test_003_1ksamples_stochastic/2_real-mse_iter_zoom.png)

### 1k samples SGD test - MSE over time
![MSE over time](media/img/tests/test_003_1ksamples_stochastic/3_mse_time.png)

### 1k samples SGD test - RMSE over time
![RMSE over time](media/img/tests/test_003_1ksamples_stochastic/3_real-mse_time.png)
![RMSE over time](media/img/tests/test_003_1ksamples_stochastic/3_real-mse_time_zoom.png)

## Theoretical point of view
### Nodes' degree
We've noticed that the iterations' speed is strongly related to the number of connections established among nodes, this is clearly observable in all _iteration over time_ plots. A similar speed gain can be seen in _MSE over time_ plot: whether the amount of samples in the training set is big enough such that even a single node alone can achieve a good solution, then connections are just cause of slowdown. It seems that the gain in iterations' speed always matters far more than the gain in information flow due to an high dependency graph's degree. This is only a suggestion given by these early tests.

Now let's consider the following plot (_MSE over time 10k samples GD test 003_).

![MSE_over_time](media/img/tests/test_003_10ksamples_classic/3_mse_time_big.png)

By inverting it we obtain the plot below. _NB: actually such functions aren't invertible since in small intervals the error can oscillate, anyway fixes can be applied to make them invertible (like exploiting a moving average)._

![Time over MSE](media/img/tests/test_003_10ksamples_classic/4_time_mse_big.png)

We can compare the time taken by each topology to achieve a same value of MSE, then effectively estimate the speed ratio among different topologies. The same will be done for RMSE/time, MSE/iter and RMSE/iter plots too. _This comparison will be included in the next report._

### Dependency graph's diameter
Assume all nodes in the system being at iteration $r>0$, assume that one node $s$ gets stuck there (it never advances to iteration $r+1$), then:
- nodes at distance 1 from $s$ can advance only by one more iteration;
- nodes at distance 2 from $s$ can advance only by two more iterations;
- and so on, up to nodes at distance $d_s$ from $s$ (where $d_s = max\{dist(s, i) : i \in V\}$) that can advance by $d_s$ more iterations.

In general:
$$
\forall i,j \in V,\ i \text{ is at iteration }r \Rightarrow j \text{ is at iteration} \in [r-dist(i,j),r+dist(i,j)] \subseteq \mathbb{N}.
$$

Since $\forall i,j \in V,\ dist(i,j) \leq diam$ (where $diam$ is the graph's diameter), then the maximum distance between two nodes, with respect to iterations' number, is always less than or equal to the graph diameter. Even this property seems to be a key factor in speed gain.

For instance, the following inequality
$$diam(\text{clique}) = 1 < diam(\text{diam-expander}) = \frac{n}{2} < diam(\text{cycle}) = n-1$$
holds also for the speed: the cycle is faster than the diam-expander which in turn is faster than the clique.

The questions now I am looking an answer for are:
1. How the speed is related to the nodes' degree?
2. How the speed is related to graph's diameter?
3. How much do degree and diameter weight in the observed topologies' speed? e.g. which matters more?

### System dynamic
Let $r \geq 0$ be a fixed iteration number. Let $X_1, X_2, ..., X_n$ be $n$ independent random variables where $X_i$ is the time taken by node $i$ to advance to iteration $r+1$. At the moment we consider $X_i \sim Exp(\lambda = 1)$, so
- $P(X_i \leq t) = 1-e^{-\lambda t} = 1-e^{-t}$, hence $P(X_i > t) = e^{-\lambda t} = e^{-t}$.
- $\mathbb{E}[X_i] = \frac{1}{\lambda} = 1$.

Lets define a new random variable $Z = max\{X_i : 1 \leq i \leq n\}$. $Z$ is the time taken by the slowest node to complete the $(r+1)$-th update of its weight vector, where:
- $P(Z \leq z) = \prod_{i=1}^{n} P(X_i \leq z) = (1-e^{-t})^n$;
- $\mathbb{E}[Z] = \frac{1}{\lambda} \sum_{i=1}^{n} \frac{1}{i}$.


$Z$ reflects the time taken by the slowest node to perform the $(r+1)$-th weight update.

It is easy to exploit $Z$ to analyze the clique behaviour since $Z$ is exactly the time taken by each node to perform the $(r+1)$-th update of its weight. For other topologies it is not so trivial. Anyway, this is the way I will follow to give significance from a theoretical point of view to results. _Further considerations will be provided in the next report._

### Iteration speed lower bound
Let $k$ be the degree of a regular dependency graph $G$. $V_l$ is the velocity lower bound in terms of iterations per node and per time, so the unit of measurement is $\frac{\text{# iterations}}{\text{time}}$.

In the worst case the node most advanced in the computation will need to wait for all its neighbours and only then it can start a new iteration.

In such case, if computations are I.I.D (independent and identically distributed) exponentials $X_1, X_2, ..., X_n$ with parameter $\lambda$, the mean time $$T_l$$ each node have to wait is

$$T = \mathbb{E}[Z] = \frac{1}{\lambda}\sum_{i=1}^{k} \frac{1}{i}$$.

The per-node iteration rate (e.g. the speed to perform a single iteration step) will be

$$V = \frac{1}{T} = \frac{\lambda}{\sum_{i=1}^{k}\frac{1}{i}}$$

so

$$V_l = \frac{\lambda}{1+\sum_{i=1}^{k}\frac{1}{i}} \sim \frac{\lambda}{\ln k} \text{ for large } k$$

is a lower bound for $V$.

#### Comparison with exact calculations for the clique ($k=n-1$)

$$V = \frac{\lambda}{\sum_{i=1}^{k}\frac{1}{i}} > \frac{\lambda}{1+\sum_{i=1}^{n-1}\frac{1}{i}} = V_l$$

#### Why is it a lower bound for $k=1$
![Example](media/img/graph-samples/iter-speed-lb-example.png)

`a` is the most advanced node (e.g. `a.iteration` $>$ `b.iteration` and `a.iteration` $>$ `c.iteration`), so it is waiting for `b` to finish its calculations, but actually `b` already got the required input from `c`, then, as soon as `b` finishes with the current computation, both `a` and `b` can move to compute to the next step. Then the next iteration will be computed by the most advanced node (it can be either `a` or `b`) after $\frac{1}{\lambda} + \frac{1}{2 \lambda}$ and not after $\frac{1}{\lambda} + \frac{1}{\lambda}$ as considered by the lower bound.